In [58]:
import numpy as np
import matplotlib.pyplot as plt

from qdc.diffuser.diffuser_sim import DiffuserSimulation, Field
from qdc.diffuser.utils import prop_farfield_fft, backprop_farfield_fft, ft2, ift2, phase_screen_diff, propagate_free_space
from qdc.diffuser.diffuser_result import DiffuserResult

plt.close('all')
show_all = False

sim = DiffuserSimulation(
    Nx=2048, Ny=2048,
    Lx=8e-3, Ly=8e-3,
    wl0=808e-9,
    Dwl=300e-9,
    N_wl=11,
    waist=40e-6,
    focal_length=200e-3,
    init_off_axis=0.2000e-6,
    diffuser_angle=0.2 * 2*np.pi/360
)

# detector plane 
field_det = sim.make_detection_gaussian(sim.wl0)
if show_all or False:
    field_det.show(title="Initial Gaussian at Detection Plane (center WL)")
print(f'{(np.abs(field_det.E) ** 2).sum()=}')


# crystal plane 
field_lens = propagate_free_space(field_det, sim.f)
field_lens.E *= sim.get_lens_mask(sim.f, sim.wl0)
field_crystal = propagate_free_space(field_lens, sim.f)

if show_all or True:
    fig, ax = plt.subplots()
    pcm = ax.imshow(np.angle(sim.get_lens_mask(sim.f, sim.wl0)), extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                    cmap='viridis', origin='lower')
    fig.colorbar(pcm, ax=ax, label='Phase [rad]')
    ax.set_title("Lens Phase (lam_center)")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")
    fig.show()


if show_all or False:
    field_crystal.show(title=f"Field Intensity at Crystal Plane f={sim.f}")

# diffuser 
field_crystal.E *= np.exp(1j * sim.diffuser_mask)
# here "switch wl", but degenerate
field_crystal.E *= np.exp(1j * sim.diffuser_mask)

if show_all or False:
    fig, ax = plt.subplots()
    pcm = ax.imshow(sim.diffuser_mask, extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                    cmap='viridis', origin='lower')
    fig.colorbar(pcm, ax=ax, label='Phase [rad]')
    ax.set_title("Single Diffuser Phase (lam_center)")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")
    fig.show()

# k-space of field after diffuser 
from qdc.diffuser.utils import ft2
G, f_x, f_y = ft2(field_crystal.E, field_crystal.x, field_crystal.y)
Ik = np.abs(G)**2
if show_all or True:
    fig, ax = plt.subplots()
    im = ax.imshow(Ik, origin='lower', cmap='viridis')
    fig.colorbar(im, ax=ax, label='|G(kx,ky)|^2')
    ax.set_title("k-space amplitude after diffuser")
    ax.set_xlabel("freq_x [index]")
    ax.set_ylabel("freq_y [index]")
    fig.show()

# final 
field_lens2 = propagate_free_space(field_crystal, sim.f)
field_lens2.E *= sim.get_lens_mask(sim.f, sim.wl0)
if show_all or True:
    field_lens2.show(title=f"Speckle at lens2")

field_det_new = propagate_free_space(field_lens2, sim.f)

if show_all or True:
    field_det_new.show(title=f"Speckle at Detection (WL={field_det_new.wl*1e9:.1f}nm)")

# twilight, inferno, viridis 



(np.abs(field_det.E) ** 2).sum()=1.0000000000000002


In [35]:
# -------------
# 7) Full Incoherent Sum Over All Wavelengths
# -------------
delta_lambdas, fields = sim.run_SPDC_simulation()  # uses the single diffuser approach inside
res = DiffuserResult()
if show_all or False:
    fig, ax = plt.subplots()
    im = ax.imshow(I_sum, origin='lower',
                   extent=[sim.x[0]*1e3, sim.x[-1]*1e3, sim.y[0]*1e3, sim.y[-1]*1e3],
                   cmap='viridis')
    fig.colorbar(im, ax=ax, label='Intensity')
    ax.set_title("Incoherent Sum of Speckle (All WLs)")
    ax.set_xlabel("x [mm]")
    ax.set_ylabel("y [mm]")
    fig.show()

print("Final result:", res)
# print("Contrast (full frame):", res.compute_contrast())

Final result: <qdc.diffuser.diffuser_result.DiffuserResult object at 0x000001F155358EE0>


In [38]:
fields[1].show()

(<Figure size 640x480 with 2 Axes>,
 <AxesSubplot: title={'center': 'Intensity (λ=809.1 nm)'}, xlabel='x [µm]', ylabel='y [µm]'>)

In [34]:
np.diff(np.array(delta_lambdas))
np.array(delta_lambdas)

array([0.00000000e+00, 1.46474197e-09, 2.92949116e-09, 4.39425478e-09,
       5.85904003e-09, 7.32385415e-09, 8.78870433e-09, 1.02535978e-08,
       1.17185418e-08, 1.31835435e-08, 1.46486101e-08, 1.61137489e-08,
       1.75789670e-08, 1.90442718e-08, 2.05096704e-08, 2.19751700e-08,
       2.34407779e-08, 2.49065013e-08, 2.63723475e-08, 2.78383236e-08,
       2.93044368e-08])